In [ ]:
#load_ext autoreload
#autoreload 2

import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
#from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import clear_output

import os, sys, glob, cv2, hdf5storage, time
import torch.nn as nn

from torchvision import transforms
import scipy.io

import models.dataset as ds
import helper as hp
#import helper as hp
import hdf5storage
import time
from skimage.metrics import structural_similarity as ssim
import models.save_loss as SL

import matplotlib as mpl
from PIL import Image
mpl.rc('image', cmap='inferno')


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = 'cuda:0'
#dtype = torch.cuda.FloatTensor

In [ ]:
model_filepath='./saved_data/trained_20240909continuous_size368_range500_newsensor_oldmask_resunet_L2L1_51/'
#model_filepath='./saved_data/trained_20250203baseline_range500_resunet_L2L1_51/'
model, args = hp.load_model(model_filepath, device = device)

In [ ]:
image_height_org = 1960 #486 #1944
image_width_org = 2592 #648 #2592
image_height_crop = 1960 #480 #1936
image_width_crop = 2592 #640 #2592
image_shift_height = 0
image_shift_width = 0#215

target_height = image_height_crop #1900#1900 #1936
target_width = image_width_crop #1900#2200 #2592

padding_x = 0 #128
padding_y = 0 #128

In [ ]:
dir_folder_cap = 'C:/Users/Huayu/Desktop/Huayu/Projects/HRME-LFOV/finetune/forwardModel/20240909-newsensor-oldmask/20250516/gain_15/'
#dir_folder_cap = 'C:/Users/Huayu/RRK Lab Dropbox/Huayu Hou/HRME-LFOV/data/casp/'
dir_folder_save = 'C:/Users/Huayu/Desktop/Huayu/Projects/HRME-LFOV/finetune/forwardModel/20240909-newsensor-oldmask/20250516/gain_15_recon/'
#dir_folder_save = 'C:/Users/Huayu/RRK Lab Dropbox/Huayu Hou/HRME-LFOV/data/casp_recon/'

if not os.path.exists(dir_folder_save):
    os.makedirs(dir_folder_save)
    
def getFileList(dir, Filelist, ext = None):
    
    newDir = dir
    if os.path.isfile(dir):
        if ext is None:
            Filelist.append(dir)
        else:
            if ext in dir[-3:]:
                Filelist.append(dir)
    elif os.path.isdir(dir):
        for s in os.listdir(dir):
            newDir = os.path.join(dir,s)
            getFileList(newDir, Filelist, ext)
    return Filelist

def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:  # MSE is zero means no noise is present in the signal.
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_ssim(img1, img2):
    ssim_value, _ = ssim(img1, img2, full=True)
    return ssim_value

imglist = getFileList(dir_folder_cap, [], 'png')
#gtlist = getFileList(dir_folder_gt, [], 'png')
print(str(len(imglist)))
#print(str(len(gtlist)))
#print(imglist.shape)

In [ ]:
#out_dir = './recon_folder_test/'
out_dir = dir_folder_save
#PSNR = np.zeros((len(imglist), 1))
#SSIM = np.zeros((len(imglist), 1))
ii = 0

for imgpath in imglist:
  #  gtpath = gtlist[ii]
    imgname = os.path.splitext(os.path.basename(imgpath))[0]
  #  gtname = os.path.splitext(os.path.basename(gtpath))[0]
    
    raw = cv2.imread(imgpath) / 255.0 
    
  # raw_gt = cv2.imread(gtpath) / 255.0 
    meas=raw[:,:,1]
    bg = np.min(raw) #np.mean(raw[3:10, 3:10]);
    print(bg);
  #  gt=raw_gt[:,:,1]
    
    measurement = meas-bg#hp.crop_images(meas, image_height_org, image_width_org, image_height_crop, image_width_crop, image_shift_height, image_shift_width)
    
   # gt = hp.crop_images(gt, image_height_org, image_width_org, image_height_crop, image_width_crop, image_shift_height, image_shift_width)
   # gt = (gt-gt.min())/(gt.max()-gt.min())
  #  gt = gt.astype('float32')/np.max(gt)
    
    meas_tensor=torch.tensor(measurement, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)#.unsqueeze(0)
    meas_pad = hp.padding(meas_tensor, padding_x, padding_y).to(device)
    
    start_time = time.time()
    out = model(meas_pad.to(device))
                
    output_numpy = out.detach().cpu().numpy()[0][0]
    del out
    
  #  output_numpy = hp.crop_out_single(output_numpy, target_height, target_width, 0, 0)
    output_numpy[output_numpy <= 0] = 0
    output_numpy = (output_numpy-output_numpy.min())/(output_numpy.max()-output_numpy.min())
    
    torch.cuda.empty_cache() 
    
  #  PSNR[ii,0] = calculate_psnr(gt, output_numpy)
  #  SSIM[ii,0] = calculate_ssim(gt, output_numpy)
    
    outname = [out_dir, imgname, '_recon.png']
    out_uint8 = (255 * output_numpy).astype(np.uint8)
    image = Image.fromarray(out_uint8)
   # plt.imsave("".join(outname), output_numpy, cmap='gray')
    image.save("".join(outname))
    
    end_time = time.time()
    print('Image #', ii, ', processing time:', end_time-start_time, end='\r')
   # SL.save_param_to_file(ii, PSNR[ii,0], SSIM[ii,0], filename = out_dir + 'resunet_Allparameters.txt')
    del start_time, end_time
    ii = ii+1

#plt.imsave('E:/FlatNet-code/MultiWienerNet-main/MultiWienerNet-main/data/real_data/simulation/00020_recon200.png', out_np, cmap='gray')

In [ ]:
#PSNR_mean = np.mean(PSNR)
#PSNR_std = np.std(PSNR)
#SSIM_mean = np.mean(SSIM)
#SSIM_std = np.std(SSIM)
#SL.save_avgparam_to_file(PSNR_mean, PSNR_std, SSIM_mean, SSIM_std, filename = filepath_save + 'resunet_Avgparameters.txt')